In [1]:
import pandas as pd
import numpy as np
from numpy import array

import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,MinMaxScaler,RobustScaler
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Binarizer,OneHotEncoder
from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import scipy
from scipy import stats,interpolate
from scipy.stats import skew,kurtosis,describe,variation,shapiro,levene,f_oneway
from scipy.special import kl_div
import  scipy.signal.signaltools

def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered

import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
def describex(data):
        data = pd.DataFrame(data)
        stats = data.describe()
        skewness = data.skew()
        kurtosis = data.kurtosis()
        skewness_df = pd.DataFrame({'skewness':skewness}).T
        kurtosis_df = pd.DataFrame({'kurtosis':kurtosis}).T
        Appended_Stats = pd.concat([stats,skewness_df,kurtosis_df],axis=0)
        return Appended_Stats

In [3]:
data1 = pd.read_csv('MEASURED_WAVE_TRAIN.csv',encoding='cp949')
df1 =  pd.DataFrame(data1.MEASURED_WAVE.str.split(',').tolist()) # 넉넉히 설정 
df1.columns = 'MEASURED_WAVE_' + df1.columns.astype(str)
data1 = pd.concat([data1[['REFERENCE_ID','QUALITY']],df1],axis=1)

data2 = pd.read_csv('REFERENCE_WAVE.csv',encoding='cp949')
df2 =  pd.DataFrame(data2.REFERENCE_WAVE.str.split(',').tolist()) # 넉넉히 설정 
df2.columns = 'REFERENCE_WAVE_' + df2.columns.astype(str)
data2 = pd.concat([data2[['REFERENCE_ID']],df2],axis=1)


for col in data1.columns:

            try : data1[col] = data1[col].astype('float')
            except: pass     
            
            
for col in data2.columns:

            try : data2[col] = data2[col].astype('float')
            except: pass     
            
print(data1.shape, data2.shape)

(27176, 202) (9590, 201)


In [4]:
data1.head()

,REFERENCE_ID,QUALITY,MEASURED_WAVE_0,MEASURED_WAVE_1,MEASURED_WAVE_2,MEASURED_WAVE_3,MEASURED_WAVE_4,MEASURED_WAVE_5,MEASURED_WAVE_6,MEASURED_WAVE_7,...,MEASURED_WAVE_190,MEASURED_WAVE_191,MEASURED_WAVE_192,MEASURED_WAVE_193,MEASURED_WAVE_194,MEASURED_WAVE_195,MEASURED_WAVE_196,MEASURED_WAVE_197,MEASURED_WAVE_198,MEASURED_WAVE_199
0,faea7610122d0747,GOOD,556.0,564.0,577.0,587.0,596.0,606.0,617.0,627.0,...,750.0,718.0,690.0,662.0,608.0,582.0,553.0,527.0,504.0,479.0
1,9f3578aa0a830cf4,GOOD,510.0,519.0,527.0,536.0,549.0,559.0,571.0,595.0,...,609.0,582.0,555.0,531.0,482.0,461.0,438.0,418.0,399.0,379.0
2,78e85aaa12ee4baf,GOOD,625.0,637.0,662.0,670.0,676.0,688.0,738.0,766.0,...,647.0,614.0,589.0,560.0,535.0,483.0,456.0,430.0,407.0,390.0
3,a92e988962a02594,GOOD,553.0,565.0,574.0,601.0,616.0,629.0,644.0,676.0,...,628.0,600.0,574.0,524.0,501.0,476.0,453.0,411.0,392.0,374.0
4,c963fce20f8d8da0,GOOD,398.0,399.0,411.0,415.0,420.0,432.0,437.0,442.0,...,432.0,392.0,374.0,356.0,322.0,308.0,296.0,270.0,256.0,243.0


In [5]:
data1['REFERENCE_ID'].value_counts()

REFERENCE_ID
0f08e3077f2767cf    302
bd39b8c9973e0190    293
e1338b3de8d158d7    270
549815b4783b17ee    254
cb6361ad4c6272fa    240
                   ... 
493b3e0850ef7128      1
c2929491cb9ccd39      1
04431c9e2fde30bb      1
8299f90b70bb3586      1
5a914c61494f3442      1
Name: count, Length: 9407, dtype: int64

In [6]:
data2.head()

,REFERENCE_ID,REFERENCE_WAVE_0,REFERENCE_WAVE_1,REFERENCE_WAVE_2,REFERENCE_WAVE_3,REFERENCE_WAVE_4,REFERENCE_WAVE_5,REFERENCE_WAVE_6,REFERENCE_WAVE_7,REFERENCE_WAVE_8,...,REFERENCE_WAVE_190,REFERENCE_WAVE_191,REFERENCE_WAVE_192,REFERENCE_WAVE_193,REFERENCE_WAVE_194,REFERENCE_WAVE_195,REFERENCE_WAVE_196,REFERENCE_WAVE_197,REFERENCE_WAVE_198,REFERENCE_WAVE_199
0,0007acd2afeb5939,523.0,531.0,539.0,558.0,568.0,581.0,592.0,611.0,622.0,...,534.0,507.0,463.0,443.0,424.0,382.0,366.0,349.0,332.0,298.0
1,00086bcac49eb5c8,788.0,802.0,818.0,837.0,853.0,870.0,886.0,906.0,926.0,...,745.0,714.0,687.0,657.0,632.0,607.0,580.0,555.0,532.0,509.0
2,000af52c92b405e3,499.0,505.0,517.0,525.0,532.0,550.0,558.0,565.0,574.0,...,605.0,556.0,530.0,506.0,461.0,439.0,419.0,400.0,364.0,348.0
3,000c2fe72c452a90,619.0,633.0,647.0,660.0,673.0,686.0,719.0,737.0,753.0,...,698.0,671.0,643.0,617.0,592.0,567.0,541.0,517.0,469.0,451.0
4,0011089305dab0f0,552.0,560.0,571.0,580.0,605.0,613.0,631.0,656.0,676.0,...,575.0,524.0,497.0,472.0,431.0,410.0,386.0,365.0,333.0,316.0


In [7]:
data2['REFERENCE_ID'].value_counts()

REFERENCE_ID
0007acd2afeb5939    1
aa39e2bc8883d56b    1
aa461b6e9e607982    1
aa4cc20f12e3f88e    1
aa5da1775a83c556    1
                   ..
53f3ab2356f936b5    1
53f4efa867bff8a3    1
53fe056fe3bb9cd7    1
5411341f6fae0882    1
fffb2d0d5a1a7552    1
Name: count, Length: 9590, dtype: int64

In [8]:
total = pd.merge(data1,data2,how='left',on = 'REFERENCE_ID').dropna()
total['QUALITY'] = total['QUALITY'].replace({'GOOD':0,'BAD':1})

In [9]:
total.shape

(27176, 402)

In [10]:
total['QUALITY'].value_counts()

QUALITY
0    25838
1     1338
Name: count, dtype: int64

In [11]:
Data = total
Target = 'QUALITY'
Input_Data = total.drop(['REFERENCE_ID','QUALITY'],axis=1)

In [12]:
def Analysis(Data,Target,Input_Data):
    
    Stats = describex(Input_Data)
    
    print('Class Count','\n',Data[Target].value_counts(),'\n')
    print('Normal', round(Data[Target].value_counts()[0]/len(Data) * 100,2), '% of the dataset')
    print('Fault', round(Data[Target].value_counts()[1]/len(Data) * 100,2), '% of the dataset')
    print('Imbalanced Ratio(IR):',round(Data[Target].value_counts()[0]/Data[Target].value_counts()[1]*1,2),'%')
    
    sns.set(font_scale=1.0)
    plt.figure(figsize=(3.5,3))
    plt.grid(True)
    plt.title('Machine  \n (0: Normal || 1: Fault)', fontsize=11)
    colors = ["blue", "red"]
    sns.countplot(x= Target, data=Data,palette=colors)
    print('\n')
    
    for col in Data.columns:
        
        try : Data[col] = Data[col].astype('float')
        except: pass  
    
    fig, ax = plt.subplots(134,3, figsize=(30, 700)) # 데이터 개수에 따라 플롯 형태 변경 가능 
    fig.subplots_adjust(hspace = .2, wspace=.3, )
    ax = ax.ravel()

    for i in range(len(Input_Data.columns)):
        ax[i] = Input_Data.iloc[:,i].plot(kind='hist', ax=ax[i],color='blue',alpha=0.5,secondary_y=True,bins=5)
        plt.title(Input_Data.columns[i])
        
    print('\n')    
    df_fault = Data[Data[Target]==1] #.set_index('TimeStamp')
    df_features = Data[Input_Data.columns] #.set_index('TimeStamp')

    ColumnNames = df_features.columns

    for feature in ColumnNames:
        sns.set_context('talk')
        _ = plt.figure(figsize=(50,8))
        _ = plt.plot(df_fault[feature], linestyle='none', marker='X', color='red', markersize=6)
        _ = plt.plot(df_features[feature], color='grey')
        _ = plt.title(feature)
        plt.show()    
        
    print('\n')    
    df_normal =  Data[Data[Target]==0][Input_Data.columns]
    df_fault = Data[Data[Target]==1][Input_Data.columns]
    
    Equal_VarTest = pd.DataFrame(stats.f_oneway(df_normal[Input_Data.columns],df_fault[Input_Data.columns])[1]).rename(columns={0:'Variance Equality F-Test '}).round(3)   
    Columns = pd.DataFrame(Data[Input_Data.columns].columns).rename(columns = {0:'Variables'})
    
    T_Statistic = pd.DataFrame(stats.ttest_ind(df_normal,df_fault,equal_var=True)[0].round(3)).rename(columns={0:'T-Statistic'})
    P_value = pd.DataFrame(stats.ttest_ind(df_normal,df_fault,equal_var=True)[1].round(3)).rename(columns={0:'P-Value'})
    df_significance = pd.concat([Columns,T_Statistic,P_value],axis=1) # 등분산성 가정 충족  
    
    T_Statistic = pd.DataFrame(stats.ttest_ind(df_normal,df_fault,equal_var=False)[0].round(3)).rename(columns={0:'T-Statistic-Welch'})
    P_value = pd.DataFrame(stats.ttest_ind(df_normal,df_fault,equal_var=False)[1].round(3)).rename(columns={0:'P-Value-Welch'})
    df_significance_Welch = pd.concat([T_Statistic,P_value],axis=1) # 등분산성 가정 위배 
    
    #print('\n')
    #plt.rcParams["figure.figsize"] = (125,100)
    #plt.title('Correlation Matrix',fontsize=25) 
    #sns.set(font_scale=1.0)
    #sns.heatmap(data = pd.concat([Data[Target],Input_Data],axis=1).corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='viridis_r')
    #plt.xticks(rotation=90) 
    
    df_corr = pd.concat([Data[Target],Input_Data],axis=1).corr().iloc[1:,0]
    df_corr = pd.DataFrame(df_corr).rename(columns={Target:'Corr'}).reset_index()['Corr']
    print('\n')
    
    X = Input_Data
    scaler = StandardScaler()
    scale_cols =  X.columns
    scaled_X = scaler.fit_transform(X[scale_cols])
    scaled_X = pd.DataFrame(scaled_X, columns=scale_cols)
    scaled_X = scaled_X.set_index(X.index)

    df_vif = pd.DataFrame()
    df_vif['VIF_Factor'] = [variance_inflation_factor(scaled_X.values, i) 
                         for i in range(scaled_X.shape[1])]
    
    Variable_Explain = pd.concat([df_significance,df_significance_Welch,Equal_VarTest,df_corr,df_vif],axis=1)
    
    print('Normality Test'.center(80))
    print('\n')
    
    for i in range(len(Input_Data.columns)):
        
        print(Input_Data.columns[i],stats.shapiro(Input_Data[Input_Data.columns[i]]),'\n')
        
    
    return  Stats, Variable_Explain #, LR_Coeff

In [13]:
#Stats, Variable_Explain = Analysis(Data,Target,Input_Data)

In [14]:
#Stats

In [15]:
#Variable_Explain

In [16]:
#Variable_Explain.sort_values(by='P-Value',ascending=True)

In [17]:
#Variable_Explain.sort_values(by='P-Value-Welch',ascending=True)

In [18]:
#Variable_Explain.sort_values(by='Corr',ascending=False)

In [19]:
#Variable_Explain.sort_values(by='VIF_Factor',ascending=True)